In [0]:
%pip install -r requirements.txt
dbutils.library.restartPython()

In [0]:
import yaml

# Load config
with open('ws_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Load environment vars and unity catalog vars 
env_vars = config['environment_variables']
unity_catalog_vars = config['unity_catalog']

In [0]:
import os 

# Set ENV vars using yaml config 
os.environ["CUDA_LAUNCH_BLOCKING"] = env_vars['CUDA_LAUNCH_BLOCKING']
os.environ['DATABRICKS_HOST'] = env_vars['DATABRICKS_HOST']
os.environ['DATABRICKS_WORKSPACE_ID'] = env_vars['DATABRICKS_WORKSPACE_ID']

# Get UC vars from config 
catalog_name = unity_catalog_vars['catalog']
schema_name = unity_catalog_vars['schema']
volume_name = unity_catalog_vars['schema']

# Get Databricks secrets information
databricks_token_scope = env_vars['DATABRICKS_TOKEN']['scope']
databricks_token_key = env_vars['DATABRICKS_TOKEN']['key']

# Set Databricks token using Databricks secrets
os.environ['DATABRICKS_TOKEN'] = db_token = dbutils.secrets.get(scope=databricks_token_scope, key=databricks_token_key)

In [0]:
catalog_name

In [0]:
# Create assets if not exist

try:
    spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
except Exception as e:
    if "quota" in str(e).lower():
        print("Quota limit reached for catalog creation.")
    else:
        raise e

# Quota limits affecting the catalog, but schema and volume should work fine. 

try: 
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
    spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.{volume_name}")
    print(f"Successfully created schema: {schema_name}, and volume: {volume_name} in catalog: {catalog_name}")
except Exception as e:
    raise e